In [1]:
import pandas as  pd
import numpy as np
import re

In [2]:
df = pd.read_csv('data/hotel_csv_data.csv')

In [3]:
df.sample(n=5)

,101,NK,1,NUID065253359,1.1,071623 113711 101,11:37 AM 07/16/2023,2,1.2,01:37 PM 07/16/2023,...,41.08.4,0.00.4,0.00.5,Unnamed: 54,F.2,F.3,Unnamed: 57,3,7.00%6.00%4.000%17.000,0000
1800,106,K,1,NUID205959455,1,081523 122708 106,12:27 PM 08/15/2023,2,1.0,02:27 PM 08/15/2023,...,40.0,0.0,0.0,NaN,F,F,NaN,1,7.00%6.00%4.000%17.000,0
4014,118,NQQ,1,NUIDE01654327,1,071923 110000 118,11:00 AM 07/19/2023,N,1.0,11:00 AM 07/20/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,7,7.00%6.00%4.000%17.000,0
5033,108,K,1,TX33963741,1,011323 082700 108,08:27 PM 01/13/2023,2,1.0,10:27 PM 01/13/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,1,7.00%6.00%4.000%17.000,0
1860,106,K,1,TX25589602,1,070923 031319 106,03:13 PM 07/09/2023,2,1.0,05:13 PM 07/09/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,2,7.00%6.00%4.000%17.000,0
5949,122,NK,1,TX11467147,1,011023 121944 122,12:19 PM 01/10/2023,N,1.0,11:00 AM 01/11/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,3,7.00%6.00%4.000%17.000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 61 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   101                     6665 non-null   int64  
 1   NK                      6665 non-null   object 
 2   1                       6665 non-null   int64  
 3   NUID065253359           6665 non-null   object 
 4   1.1                     6665 non-null   object 
 5   071623 113711 101       6665 non-null   object 
 6   11:37 AM   07/16/2023   6665 non-null   object 
 7   2                       6663 non-null   object 
 8   1.2                     6663 non-null   float64
 9   01:37 PM   07/16/2023   6665 non-null   object 
 10  CreditCard              6665 non-null   object 
 11  F                       6665 non-null   object 
 12  41.08                   6665 non-null   float64
 13  Unnamed: 13             140 non-null    object 
 14  0.00                    6665 non-null   

In [5]:
idx_to_keep = [0, 1 ,6, 7, 8, 9, 12, 21, 23]

In [6]:
# create new dataframe with specified column indices
df1 = df.iloc[:, idx_to_keep].copy()

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   101                    6665 non-null   int64  
 1   NK                     6665 non-null   object 
 2   11:37 AM   07/16/2023  6665 non-null   object 
 3   2                      6663 non-null   object 
 4   1.2                    6663 non-null   float64
 5   01:37 PM   07/16/2023  6665 non-null   object 
 6   41.08                  6665 non-null   float64
 7   35.11                  6665 non-null   float64
 8   5.97                   6665 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 468.8+ KB


In [8]:
# Create a dictionary to rename the columns
column_names = ['room_num', 
                'room_type', 
                'checkin_timestamp',
                'stay_duration',
                'num_nights',
                'checkout_timestamp',
                'total_price',
                'price',
                'tax']

# Assign column names to the DataFrame
df1.columns = column_names

In [9]:
df1.sample(n=5)

,room_num,room_type,checkin_timestamp,stay_duration,num_nights,checkout_timestamp,total_price,price,tax
3574,101,NK,08:51 PM 06/11/2023,6,1.0,02:51 AM 06/12/2023,65.0,55.56,9.44
5335,106,K,11:47 PM 01/08/2023,N,1.0,11:00 AM 01/09/2023,70.0,59.83,10.17
901,122,NK,01:20 AM 09/11/2023,N,1.0,11:00 AM 09/11/2023,65.0,55.56,9.44
2826,102,NK,10:35 PM 11/21/2022,N,1.0,11:00 AM 11/22/2022,70.0,59.83,10.17
164,116,K,5/19/2023 3:00:00 PM,1,1.0,5/19/2023 4:00:00 PM,25.0,21.37,3.63


In [10]:
# Convert checkin_timestamp and checkout_timestamp
def convert_string_to_datetime(df, column_name, new_column_name):
  # Create a datetime object from the string object.
  datetime_object = pd.to_datetime(df[column_name], errors='coerce', format='mixed')

  # Add the datetime object to the DataFrame as a new column.
  new_datetime_column = f'{new_column_name}_datetime'
  df[new_datetime_column] = datetime_object

  # drop old timestamp columns
  df = df.drop([column_name], axis=1)

  return df



In [11]:
df1 = convert_string_to_datetime(df1.copy(), 'checkin_timestamp', 'checkin')
df1 = convert_string_to_datetime(df1.copy(), 'checkout_timestamp', 'checkout')


In [13]:
# drop old timestamp columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   room_num           6665 non-null   int64         
 1   room_type          6665 non-null   object        
 2   stay_duration      6663 non-null   object        
 3   num_nights         6663 non-null   float64       
 4   total_price        6665 non-null   float64       
 5   price              6665 non-null   float64       
 6   tax                6665 non-null   float64       
 7   checkin_datetime   6665 non-null   datetime64[ns]
 8   checkout_datetime  6665 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 468.8+ KB
